Python 連線 PostgreSQL 資料庫測試

In [ ]:
#導入postgresql套件
import psycopg2
#環境變數儲存器(可先將環境變數儲存在.env的檔案中，透過load_dotenv將環境變數導入)
from dotenv import load_dotenv
#呼叫系統檔案或程式
import os
load_dotenv()

#psycopg2.connect建立DB連線，os.environ取得相關環境變數
conn = psycopg2.connect(host=os.environ['POSTGRE_HOST'],database=os.environ['POSTGRE_DATABASE'],user=os.environ['POSTGRE_USER'],password=os.environ['POSTGRE_PASSWORD'])
#開啟 with 管理資源分配器(Context Manager)，DB連線時開啟，結束時關閉
with conn:
    print("Opened database successfully")

Opened database successfully


建立資料表

In [ ]:
#導入postgresql套件
import psycopg2
#環境變數儲存器(可先將環境變數儲存在.env的檔案中，透過load_dotenv將環境變數導入)
from dotenv import load_dotenv
#呼叫系統檔案或程式
import os
load_dotenv()

#psycopg2.connect建立DB連線，os.environ取得相關環境變數
conn = psycopg2.connect(host=os.environ['POSTGRE_HOST'],database=os.environ['POSTGRE_DATABASE'],user=os.environ['POSTGRE_USER'],password=os.environ['POSTGRE_PASSWORD'])
#開啟 with 管理資源分配器(Context Manager)，DB連線時開啟，結束時關閉
with conn:
    with conn.cursor() as cursor:
        sql = '''CREATE TABLE IF NOT EXISTS 市場(
        name VARCHAR(20),
        country VARCHAR(20),
        PRIMARY KEY(name),
        UNIQUE(name,country) /*不可以有重覆的資料*/
        );'''

        cursor.execute(sql)
    with conn.cursor() as cursor:
        sql = '''CREATE TABLE IF NOT EXISTS 股市(
            stock_id SERIAL,
            date DATE,
            open NUMERIC(17,10),
            high NUMERIC(17,10),
            low NUMERIC(17,10),
            close NUMERIC(17,10),
            adj_close NUMERIC(17,10),
            volume BIGINT DEFAULT 0,
            name VARCHAR(20),
            PRIMARY KEY(stock_id),	
            UNIQUE(date,name),
            FOREIGN KEY(name) REFERENCES 市場(name)
            ON DELETE NO ACTION
            ON UPDATE CASCADE
        );'''
        cursor.execute(sql)

新增資料

In [ ]:
#導入postgresql套件
import psycopg2
#環境變數儲存器(可先將環境變數儲存在.env的檔案中，透過load_dotenv將環境變數導入)
from dotenv import load_dotenv
#呼叫系統檔案或程式
import os
load_dotenv()

#psycopg2.connect建立DB連線，os.environ取得相關環境變數
conn = psycopg2.connect(host=os.environ['POSTGRE_HOST'],database=os.environ['POSTGRE_DATABASE'],user=os.environ['POSTGRE_USER'],password=os.environ['POSTGRE_PASSWORD'])
#開啟 with 管理資源分配器(Context Manager)，DB連線時開啟，結束時關閉
with conn:
    with conn.cursor() as cursor:
        sql='''
        INSERT INTO 市場 values (%s,%s)
        ON CONFLICT DO NOTHING;
        '''
        cursor.execute(sql,['^TWII','台灣'])
        cursor.execute(sql,['^HSI','香港'])

查詢資料

In [ ]:
#導入postgresql套件
import psycopg2
#環境變數儲存器(可先將環境變數儲存在.env的檔案中，透過load_dotenv將環境變數導入)
from dotenv import load_dotenv
#呼叫系統檔案或程式
import os
load_dotenv()

#psycopg2.connect建立DB連線，os.environ取得相關環境變數
conn = psycopg2.connect(host=os.environ['POSTGRE_HOST'],database=os.environ['POSTGRE_DATABASE'],user=os.environ['POSTGRE_USER'],password=os.environ['POSTGRE_PASSWORD'])
# 開啟 with 管理資源分配器(Context Manager)，DB連線時開啟，結束時關閉
with conn:
    with conn.cursor() as cursor:
        sql='''
        SELECT name,country FROM 市場
        '''
        cursor.execute(sql)
        rows = cursor.fetchall()
        for row in rows:
            print("Name = ",row[0])
            print("Country = ",row[1],'\n')

Name =  ^TWII
Country =  台灣 

Name =  ^HSI
Country =  香港 



修改資料

In [ ]:
#導入postgresql套件
import psycopg2
#環境變數儲存器(可先將環境變數儲存在.env的檔案中，透過load_dotenv將環境變數導入)
from dotenv import load_dotenv
#呼叫系統檔案或程式
import os
load_dotenv()

#psycopg2.connect建立DB連線，os.environ取得相關環境變數
conn = psycopg2.connect(host=os.environ['POSTGRE_HOST'],database=os.environ['POSTGRE_DATABASE'],user=os.environ['POSTGRE_USER'],password=os.environ['POSTGRE_PASSWORD'])
#開啟 with 管理資源分配器(Context Manager)，DB連線時開啟，結束時關閉
with conn:
    with conn.cursor() as cursor:
        #因為修改資料後馬上要顯示出修改的結果，所以語法中要加入 RETURNING
        sql = '''
        UPDATE 市場 SET name = '^TWII' WHERE Country = '台灣' RETURNING name,Country;
        '''
        cursor.execute(sql)
        rows = cursor.fetchall()
        for row in rows:
            print("Name = ",row[0])
            print("Country = ",row[1],'\n')

Name =  ^TWII
Country =  台灣


當有外部鍵(foreign key)時如何清空資料表

In [ ]:
TRUNCATE TABLE 資料表 RESTART IDENTITY CASCADE;

刪除資料